# Питон и машинное обучение

# Модуль 2. Предварительная обработка данных

- Добавление, удаление, визуализация признаков
- Нормализация и шкалирование данных
- Преобразования признаков
- Кодирование признаков



In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn import datasets

from scipy import stats 

%matplotlib inline

In [ ]:
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('always', category=UserWarning)

### Работа с признаками в Pandas

Здесь мы рассмотрим базовые операции по предобработке данных в контексте отдельных признаков датасета.

Предварительный анализ признаков лучше делать на полном датасете, который включает в себя и определяющую, и результирующую части.

Более подробно про модификацию признаков будет рассказано по ходу курса.

In [ ]:
# снова загрузим "ирисы"
iris_raw = datasets.load_iris()
iris = pd.DataFrame(iris_raw.data, columns=iris_raw.feature_names)
iris['y'] = iris_raw.target
iris

Переименование признаков:

In [ ]:
iris.columns = ['sepal_l', 'sepal_w', 'petal_l', 'petal_w', 'y']
iris

Посмотреть статистические характеристики для всех числовых признаков можно при помощи функции ```describe()```:

In [ ]:
iris.describe()

Получить данные по конкретному признаку можно, обратившись к нему как к элементу словаря, это даст объект ```pd.Series``` с соответствующей колонкой из загруженной таблицы:

In [ ]:
iris['sepal_l'] 

In [ ]:
iris.sepal_l # такой способ обращения тоже поддерживается

Чтобы обратиться к каким-либо важным, на ваш взгляд, характеристикам датасета, можно использовать булевы маски.

Например, вот так можно посмотреть все ирисы, у которых ```sepal_l``` больше медианного значения:

In [ ]:
iris[ iris.sepal_l > iris.sepal_l.median() ]

Для подсчета уникальных значений категориальных признаков удобно пользоваться функцией ```value_counts()```. 

Например, вот так можно узнать распределение классов для нашего среза из предыдущей клетки:

In [ ]:
iris[ iris.sepal_l > iris.sepal_l.median() ].y.value_counts()

#### Визуализация признаков

Матрица диаграмм рассеяния, по диагонали - гистограммы распределения значений по каждому признаку.

In [ ]:
_ = pd.plotting.scatter_matrix(iris, 
                  figsize=(15, 15))

In [ ]:
!conda install -y seaborn

In [ ]:
import seaborn as sns

_ = sns.pairplot(iris,
             diag_kind='kde', plot_kws={'alpha': 0.2})

Корелляционная матрица:

In [ ]:
plt.figure(figsize=(10,10))

_ = sns.heatmap(iris.corr(), annot=True, cmap="RdYlGn")
iris.corr()

Визуализация отдельных признаков:

In [ ]:
iris['sepal_w'].hist()

In [ ]:
_ = sns.histplot(iris['sepal_w'], kde=True)

#### Добавление и удаление признаков

Вместо пары кореллирующих признаков ```petal_l``` и ```petal_w``` сделаем новый признак ```petal_avg```:

In [ ]:
iris_ = iris.copy()

iris_['petal_avg'] = (iris_.petal_l + iris_.petal_w) / 2
iris_.drop(['petal_l', 'petal_w'], axis=1, inplace=True)
iris_

In [ ]:
_ = sns.heatmap(iris_[['sepal_l','sepal_w','petal_avg','y']].corr(), annot=True, cmap="RdYlGn")

In [ ]:
_ = sns.pairplot(iris_[['sepal_l','sepal_w','petal_avg','y']], diag_kind='kde', plot_kws={'alpha': 0.2})

## Немного теории: виды распределений вероятностей

Чаще всего приходится иметь дело со следующими видами распределений:
- __Нормальное (Гауссово)__ - идеальный случай для большинства задач, соответствует ЦПТ, и т.д.
- __Логнормальное__ - распределение, которое приводится к нормальному после логарифмирования величин. Характерно для следующих данных:
    - доходы физических и юридических лиц
    - количество комментариев под постами в соц. сетях и интернет-магазинах
    - другое...
- __Бернулли__, __Пуассона__ - распределение вероятностей бинарной или дискретной случайной величины соответсвенно.
- __Бета-распределение__ - используется для предсказания вероятностей, параметры ```a``` - мера успеха, ```b``` - неуспеха, в сопряжении с распределением Бернулли.


In [ ]:
data_norm = np.random.normal(size=1000)
data_lognorm = np.random.lognormal(size=1000)
data_beta = pd.DataFrame({'a=0.5, b=0.5': np.random.beta(a=0.5, b=0.5, size=1000),
                          'a=1, b=1': np.random.beta(a=1, b=1, size=1000),
                          'a=2, b=8': np.random.beta(a=2, b=8, size=1000),
                          'a=50, b=50': np.random.beta(a=10, b=10, size=1000),
                         })

fig, axs = plt.subplots( 1, 3 )
fig.set_size_inches( (15, 5) )
axs[0].set_title('Нормальное (Гауссово)')
axs[1].set_title('Логнормальное')
axs[2].set_title('Бета')

sns.histplot(data_norm, ax=axs[0], kde=True)
sns.histplot(data_lognorm, ax=axs[1], kde=True)
sns.histplot(data_beta, ax=axs[2], kde=True)


plt.show()

Как понять на сколько мы близки к нормальному распределению? Как можно это измерить?

Можно использовать оценки Шапиро-Уилка и Колмогорова-Смирнова. ```pvalue``` в обоих случаях - это вероятность нуль-гипотезы "данные распрпделены нормально". То есть чем выше значение ```pvalue```, тем ближе к нормальному распределение анализируемых данных. 

In [ ]:
print(stats.shapiro(data_norm))
print()
print(stats.shapiro(data_lognorm))
print()
print(stats.shapiro(data_beta['a=50, b=50'])) 
print()
print(stats.shapiro(data_beta['a=1, b=1']))

In [ ]:
print(stats.kstest(data_norm, 'norm'))
print()
print(stats.kstest(data_lognorm, 'norm'))
print()
print(stats.kstest(data_beta['a=50, b=50'], 'norm')) 
print()
print(stats.kstest(data_beta['a=1, b=1'], 'norm'))

‼️ ВАЖНО! Для проверки данных на "нормальность" тестом Колмогорова-Смирнова их нужно шкалировать (приводить к распределению с мат. ожиданием в 0).

In [ ]:
from sklearn.preprocessing import StandardScaler

print( stats.kstest(StandardScaler().fit_transform(pd.DataFrame(data_beta['a=50, b=50']))[0], 'norm') )

#### Логнормальное распределение

In [ ]:
_ = sns.histplot(data_lognorm, kde=True)

In [ ]:
fig, axs = plt.subplots( 2, 1 )
fig.set_size_inches( (8, 8) )
axs[0].set_title('Логнормальное')
axs[1].set_title('После логарифмирования')

sns.histplot(data_lognorm, ax=axs[0], kde=True)
sns.histplot(np.log10(data_lognorm), ax=axs[1], kde=True)


plt.show()

print(stats.shapiro(data_lognorm))
print()
print(stats.shapiro(np.log10(data_lognorm)))

### Мультимодальное распределение



In [ ]:
dist_1 = np.random.normal(10, 3, 1000)
dist_2 = np.random.normal(30, 5, 4000)
dist_3 = np.random.normal(45, 6, 800)

multimodal_dist = np.concatenate((dist_1, dist_2, dist_3), axis=0)

_ = sns.histplot(multimodal_dist, kde=True)

Вычислить моды позволяет модель ```GaussianMixture```.

In [ ]:
from sklearn.mixture import GaussianMixture

gmm = GaussianMixture(n_components=3)
gmm.fit(multimodal_dist.reshape(-1, 1))

means = gmm.means_.flatten()

# сигмы вычисляем по квадратным корням ковариантностей
standard_deviations = ( gmm.covariances_**0.5 ).flatten()

# пропорции данных в распределениях
weights = gmm.weights_.flatten()


print(f"Means: {means},\n\
Standard Deviations: {standard_deviations}")

### Преобразования признаков

1. Min-max шкалирование:


$\tilde{x}_i = \frac{x_i - x_{min}}{x_{max} - x_{min}}$

Позволяет поместить данные в диапазон ```[0, 1]```.

In [ ]:
from sklearn.preprocessing import MinMaxScaler

iris_minmax = pd.DataFrame( MinMaxScaler().fit_transform(iris.iloc[ :, :-1 ]), columns = iris.columns[:-1] )
iris_minmax.describe()

2. Стандартное шкалирование: нормализация с мат. ожиданием в 0 и дисперсией 1:

$\tilde{x}_i = \frac{x_i - \mu}{\sigma}$

Не приводит данные к нормальному распределению, но позволяет "сгладить" выбросы.

In [ ]:
from sklearn.preprocessing import StandardScaler

iris_std = pd.DataFrame( StandardScaler().fit_transform(iris.iloc[ :, :-1 ]), columns = iris.columns[:-1] )
iris_std.describe()

3. L2-нормализация: каждый вектор делится на свой модуль:

$\tilde{x} = \frac{x}{||x||}$

"Горизонтальная" нормализация - нормализуется каждый вектор из набора данных (приводится к модулю = 1).

In [ ]:
from sklearn.preprocessing import normalize

iris_norm = pd.DataFrame( normalize(iris.iloc[ :, :-1 ]), columns = iris.columns[:-1] )
iris_norm.describe()

Построим матрицы диаграмм рассеяния и корелляции для различных способов шкалирования и нормализации:

In [ ]:
_ = sns.pairplot(iris_std,
             diag_kind='kde', plot_kws={'alpha': 0.2})

In [ ]:
_ = sns.heatmap(iris.corr(), annot=True, cmap="RdYlGn")

Подробнее о различных способах нормализации можно прочитать в [официальной документации ```scikit-learn```](https://scikit-learn.org/stable/auto_examples/preprocessing/plot_all_scaling.html).

#### ⁉️ Задание

1. В датасете "Ирисы" проверьте признак ```sepal_w``` на соответствие нормальному распределению.
2. В загруженном датасете "Ирисы" вычислите моды распределения по признакам ```petal_l``` и ```petal_w``` и сравните их.

In [ ]:
# ваш код здесь


### Обработка нестандартных распределений и "выбросов"



In [ ]:
import json

biz_f = open('data/yelp_academic_dataset_business.json')
biz_df = pd.DataFrame([json.loads(x) for x in biz_f.readlines()])
biz_f.close()

biz_df

In [ ]:
df_reviews = pd.DataFrame({'review_count': biz_df['review_count']})

sns.set_style('whitegrid')
fig, ax = plt.subplots()
df_reviews['review_count'].hist(ax=ax, bins=100)
ax.set_yscale('log')
ax.set_xlabel('Review Count')
ax.set_ylabel('Occurrence')
plt.show()

Выполним логарифмирование и шкалирование, посмотрим как изменилась гистограмма:

In [ ]:
sns.set_style('whitegrid')
fig, axs = plt.subplots(4, 1)

fig.set_size_inches( (8, 12) )
axs[0].set_title('Оригинальное распределение')
axs[1].set_title('Шкалированное распределение')
axs[2].set_title('Логарифмированное распределение')
axs[3].set_title('Логарифмированное и затем шкалированное распределение')

df_reviews_std = pd.DataFrame(StandardScaler().fit_transform(df_reviews), columns=df_reviews.columns) 
df_reviews_log = pd.DataFrame(np.log10(df_reviews), columns=df_reviews.columns)
df_reviews_log_std = pd.DataFrame(StandardScaler().fit_transform(df_reviews_log), columns=df_reviews.columns) 


df_reviews['review_count'].hist(ax=axs[0], bins=100)
df_reviews_std['review_count'].hist(ax=axs[1], bins=100)
df_reviews_log['review_count'].hist(ax=axs[2], bins=100)
df_reviews_log_std['review_count'].hist(ax=axs[3], bins=100)

for i in range(4):
    axs[i].set_yscale('log')
    

plt.show()

### Бининг🗑️, квантилизация и степенные преобразования

Бининг по заданным значениям:

In [ ]:
bins = [0,500,1000,10000]

labels = [f"rw_{border}_{bins[i+1]}" for i, border in enumerate(bins[:-1]) ]

pd.get_dummies( pd.cut(df_reviews['review_count'], bins=bins, labels=labels) )

Квантилизация по децилям:

In [ ]:
deciles = df_reviews['review_count'].quantile([.1, .2, .3, .4, .5, .6, .7, .8, .9])
deciles

In [ ]:
fig, ax = plt.subplots()
df_reviews['review_count'].hist(ax=ax, bins=100)
for pos in deciles:
    handle = plt.axvline(pos, color='r')
ax.set_yscale('log')
ax.set_xscale('log')
ax.set_xlabel('Review Count')
ax.set_ylabel('Occurrence')
plt.show()

In [ ]:
deciles_ = pd.concat([pd.Series([0.0]), deciles, pd.Series([1.0e5])])

labels = [f"{i+1}: {v:.0f}-{deciles_.iloc[i+1]:.0f}" for i, (q, v) in enumerate(deciles_.iloc[:-1].items()) ]

pd.get_dummies( 
    pd.cut(df_reviews['review_count'], bins=deciles_, labels=labels), 
    prefix='q', 
    dtype=np.int64
)

In [ ]:
pd.qcut(df_reviews['review_count'], 4, labels=False)

In [ ]:
_ = sns.histplot(pd.qcut(df_reviews['review_count'], 10, labels=False))

In [ ]:
pd.get_dummies( 
    pd.qcut(df_reviews['review_count'], 10, labels=False), 
    prefix='q', 
    dtype=np.int64
)

Другой способ квантильных преобразований:

In [ ]:
from sklearn.preprocessing import QuantileTransformer

qt = QuantileTransformer(output_distribution='normal', random_state=0)

_ = sns.histplot( qt.fit_transform(data_lognorm.reshape(-1,1)))

In [ ]:
_, ax = plt.subplots()
ax.set_yscale('log')
_ = sns.histplot( qt.fit_transform(df_reviews), ax=ax, legend=None )

Степенные преобразования, преобразование Бокса-Кокса:

$y^{(\lambda )}_i =\begin{cases}\frac{y^\lambda_i-1}{\lambda}&\lambda \neq 0\cr \ln(y) &\lambda =0\end{cases}.$

и Йео-Джонсона:

$y^{(\lambda )}_i =\begin{cases}[{(y_i + 1)^\lambda-1}]/{\lambda} &\lambda \neq 0, y_i \geq 0 \cr\
\ln(y)&\lambda = 0, y_i \geq 0 \cr\
-[(-y_i + 1)^{(2-\lambda)} - 1]/(2 - \lambda) &\lambda \neq 2, y_i < 0 \cr\
-\ln(-y_i+1）&\lambda = 2, y_i < 0
\end{cases}.$

In [ ]:
from sklearn.preprocessing import PowerTransformer

pt = PowerTransformer(method='box-cox')
# pt = PowerTransformer(method='yeo-johnson')

_ = sns.histplot( pt.fit_transform(data_lognorm.reshape(-1, 1)), legend=None )



In [ ]:
box_cox_transformer = pt.fit(df_reviews)

print(box_cox.lambdas_)

_, ax = plt.subplots()
ax.set_yscale('log')
_ = sns.histplot(box_cox_transformer.transform(df_reviews), ax=ax)

#### ⁉️ Задание

Загрузите датасет ```data/credit_scoring.csv``` и проанализируйте признак ```Income```:

1. Избавьтесь от NaN, объясните почему вы выбрали тот или иной способ избавления от пропусков
2. Приведите данные в Income к нормальному распределению. Пришлите статистику и pvalue сравнения Income с нормальным распределением по Шапиро-Уилка

In [ ]:
df = pd.read_csv('data/credit_scoring.csv', index_col='client_id')
df

In [ ]:
# ваш код здесь




## Кодирование признаков

Многие алгоритмы машинного обучения не могут работать с текстовой информацией.

Такая информация может быть либо переведена в dummy-признаки, либо закодирована.



In [ ]:
bank = pd.read_csv('data/bank.csv')
bank

Рассмотрим на примере признаков ```job``` и ```education```:

In [ ]:
print(bank.job.value_counts())
print(bank.education.value_counts())

Чтобы не сильно увеличивать размерность данных, предлагается признаки с небольшим количеством значений перевести в one-hot encoding и dummy-признаки, а признаки с большим количеством - закодировать:

In [ ]:
from sklearn.preprocessing import OrdinalEncoder

enc = OrdinalEncoder()

bank['job_encoded'] = enc.fit_transform(bank.job.values.reshape(-1,1))

bank.job_encoded.value_counts()


In [ ]:
df_edu = pd.get_dummies( bank.education, dtype=np.int64, prefix='education' )
df_edu

In [ ]:
bank.join(df_edu)

Можно закодировать весь датасет, но лучше сделать это только для текстовых признаков:

In [ ]:
text_features = bank.columns[ bank.dtypes == 'object']

enc = OrdinalEncoder()

a = enc.fit_transform(bank[ text_features ])
print(a.shape)
a

In [ ]:
# делаем из него dataframe:
df_encoded = pd.DataFrame(a, columns=text_features)

In [ ]:
# проверим данные
df_encoded.job.value_counts()

Также в ```sklearn``` реализован one-hot encoding:

In [ ]:
from sklearn.preprocessing import OneHotEncoder

enc = OneHotEncoder()

enc.fit(bank[ text_features ])

a = enc.transform(bank[ text_features ]).toarray()
print(a.shape)
a

In [ ]:
# как вернуть названия признаков?
enc.categories_

In [ ]:
columns = []
for i, text_feature in enumerate(text_features):
    for value in enc.categories_[i]:
        columns += [f"{text_feature}_{value}"]
df_one_hot = pd.DataFrame(a, columns=columns)
df_one_hot   

#### ⁉️ Задание

Для данного датасета ```bank.csv``` выполните следующее:

1. Закодируйте бинарные признаки в 0 и 1 (1=истина).
1. Закодируйте те признаки, в которых не более четырех значений методом one-hot.
2. Закодируйте названия месяцев в числа 1-12 в соответствии месяцу.
3. Все остальные текстовые признаки закодируйте методом OrdinalEncoding.

In [ ]:
# ваш код здесь

